<a href="https://colab.research.google.com/github/matthewincledon/Project-Files/blob/master/Matthew_Incledon_Data_Platform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Your colleague is working on a game to help teach kids about the solar system. She found an API that would let her get information about solar system objects. Using the [API documentation](https://api.le-systeme-solaire.net/en/) and [web UI](https://api.le-systeme-solaire.net/swagger/#/bodies/get_bodies) she was able to get the data about objects classified as planets into a pandas dataframe, as shown in the code below. (Note that she had to exclude the "rel" field, as that caused a problem for the JSON->Dataframe conversion.) She is asking you to take on some of this project.

In the game, players will be able to design their own solar system objects. They can specify some of the values and leave others blank for the game to fill in with something reasonable.

To support this, please write a function that takes in a dictionary of fields and values and returns the item in the database that is "closest" to these values, so that your colleague can use it to fill in missing fields. Use your judgement about which fields you should support and which are not things that users would be choosing.

In [ ]:
import requests
import json
!pip install fsspec
import pandas as pd
import numpy as np

     |████████████████████████████████| 102kB 3.8MB/s 


In [ ]:
r = requests.get("https://api.le-systeme-solaire.net/rest.php/bodies?exclude=rel&filter%5B%5D=isPlanet%2Cneq%2Cfalse")
# GET request to this endpoint with following parameters:
# exclude = rel
# filter[] = isPlanet != FALSE
# returns 13 planet records
r.text

'{"bodies":[{"id":"ceres","name":"(1) Cérès","englishName":"1 Ceres","isPlanet":true,"moons":null,"semimajorAxis":413690250,"perihelion":382620000,"aphelion":445410000,"eccentricity":0.07582,"inclination":10.59300,"mass":{"massValue":9.39300,"massExponent":20},"vol":{"volValue":4.21000,"volExponent":9},"density":2.16100,"gravity":0.28000,"escape":510.00000,"meanRadius":476.20000,"equaRadius":487.00000,"polarRadius":455.00000,"flattening":0.96190,"dimension":"","sideralOrbit":1681.63000,"sideralRotation":9.07000,"aroundPlanet":null,"discoveredBy":"Giuseppe Piazzi","discoveryDate":"01/01/1801","alternativeName":"A899 OF - 1943 XB","axialTilt":3},{"id":"eris","name":"(136199) Éris","englishName":"136199 Eris","isPlanet":true,"moons":[{"moon":"Dysnomie"}],"semimajorAxis":10180122852,"perihelion":5765732799,"aphelion":14594512904,"eccentricity":0.44068,"inclination":44.04450,"mass":{"massValue":1.66000,"massExponent":22},"vol":{"volValue":6.59000,"volExponent":9},"density":2.52000,"gravity"

In [ ]:
df = pd.read_json(json.dumps(json.loads(r.text)["bodies"]), orient="records")
df

,id,name,englishName,isPlanet,moons,semimajorAxis,perihelion,aphelion,eccentricity,inclination,mass,vol,density,gravity,escape,meanRadius,equaRadius,polarRadius,flattening,dimension,sideralOrbit,sideralRotation,aroundPlanet,discoveredBy,discoveryDate,alternativeName,axialTilt
0,ceres,(1) Cérès,1 Ceres,True,None,413690250,382620000,445410000,0.07582,10.59300,"{'massValue': 9.393, 'massExponent': 20}","{'volValue': 4.21, 'volExponent': 9}",2.1610,0.280,510,476.2000,487.0000,455.0,0.96190,,1681.630,9.0700,NaN,Giuseppe Piazzi,01/01/1801,A899 OF - 1943 XB,3.0000
1,eris,(136199) Éris,136199 Eris,True,[{'moon': 'Dysnomie'}],10180122852,5765732799,14594512904,0.44068,44.04450,"{'massValue': 1.6600000000000001, 'massExponen...","{'volValue': 6.59, 'volExponent': 9}",2.5200,0.820,1380,1163.0000,0.0000,0.0,0.00000,,203830.000,25.9200,NaN,"Michael E. Brown, Chadwick Trujillo, David L. ...",05/01/2005,2003 UB313,0.0000
2,uranus,Uranus,Uranus,True,"[{'moon': 'Ariel'}, {'moon': 'Umbriel'}, {'moo...",2870658186,2734998229,3006318143,0.04570,0.77200,"{'massValue': 8.68127, 'massExponent': 25}","{'volValue': 6.833, 'volExponent': 13}",1.2700,8.870,21380,25362.0000,25559.0000,24973.0,0.02293,,30685.400,-17.2400,NaN,William Herschel,13/03/1781,,97.7700
3,pluton,Pluton,Pluto,True,"[{'moon': 'Charon'}, {'moon': 'Nix'}, {'moon':...",5906440628,4436756954,7376124302,0.24880,17.16000,"{'massValue': 1.303, 'massExponent': 22}","{'volValue': 7.15, 'volExponent': 9}",1.8900,0.620,1210,1188.3000,1188.3000,1195.0,0.00000,,90465.000,-153.2928,NaN,Clyde W. Tombaugh,18/02/1930,,122.5000
4,neptune,Neptune,Neptune,True,"[{'moon': 'Triton'}, {'moon': 'Néreïde'}, {'mo...",4498396441,4459753056,4537039826,0.01130,1.76900,"{'massValue': 1.02413, 'massExponent': 26}","{'volValue': 6.254, 'volExponent': 13}",1.6380,11.150,23560,24622.0000,24764.0000,24341.0,0.01708,,60189.000,16.1100,NaN,"Urbain Le Verrier, John Couch Adams, Johann Galle",23/09/1846,,28.3000
5,haumea,(136108) Hauméa,136108 Haumea,True,"[{'moon': 'Namaka'}, {'moon': 'Hi'iaka'}]",6432011461,5157623774,7706399149,0.19126,28.19000,"{'massValue': 4.006, 'massExponent': 21}","{'volValue': 2.4, 'volExponent': 9}",2.6000,0.401,809,33.0000,0.0000,498.0,0.61020,1960 × 1518 × 996,103774.000,3.9200,NaN,"Michael E. Brown, Jos&eacute; Luis Ortiz Moreno",28/12/2004,2003 EL61,0.0000
6,makemake,(136472) Makémaké,136472 Makemake,True,[{'moon': 'S/2015 (136472) 1'}],6783345606,5671928586,7894762625,0.15586,29.00685,"{'massValue': 4.4, 'massExponent': 21}","{'volValue': 1.7000000000000002, 'volExponent'...",1.4000,0.500,800,725.0000,745.0000,715.0,0.05000,,112897.000,0.0000,NaN,"Michael E. Brown, Chadwick Trujillo, David L. ...",31/03/2005,2005 FY9,0.0000
7,jupiter,Jupiter,Jupiter,True,"[{'moon': 'Io'}, {'moon': 'Europe'}, {'moon': ...",778340821,740379835,816620000,0.04890,1.30400,"{'massValue': 1.89819, 'massExponent': 27}","{'volValue': 1.43128, 'volExponent': 15}",1.3262,24.790,60200,69911.0000,71492.0000,66854.0,0.06487,,4332.589,9.9250,NaN,,,,3.1200
8,mars,Mars,Mars,True,"[{'moon': 'Phobos'}, {'moon': 'Deïmos'}]",227943824,206700000,249200000,0.09350,1.85000,"{'massValue': 6.41712, 'massExponent': 23}","{'volValue': 1.6318000000000001, 'volExponent'...",3.9341,3.710,5030,3389.5000,3396.1900,3376.2,0.00589,,686.980,24.6229,NaN,,,,25.1900
9,mercure,Mercure,Mercury,True,None,57909227,46001200,69816900,0.20560,7.00000,"{'massValue': 3.30114, 'massExponent': 23}","{'volValue': 6.083, 'volExponent': 10}",5.4291,3.700,4250,2439.4000,2440.5300,2439.7,0.00000,,87.969,1407.6000,NaN,,,,0.0352


In [ ]:
df2=df[df['id']!='ceres']
#df2[['id','englishName','density']]

###The following are supported input data points and explanation


|Field                | Explanation                |
|---------------------|:---------------------------|
|id                   |not valid, cannot know ID from API ahead of time|
|name                 | not valid, the French name of a planet being similar does not give an indication of other similar attributes|
|englishName          |not valid, the English name of a planet being similar does not give an indication of other similar attributes|
|isPlanet             |not valid, these will all be planets as input|
|moons                |not valid, the moons of a planet do not give an indication of other similar attributes|
|semimajorAxis        |valid, feature of planet|
|perihelion           |valid, describes the planets distance to the sun at closest point, which is distinct feature|
|aphelion             |valid, describes the planets distance to the sun at furthest point, which is distinct feature|
|eccentricity         |valid, distinct shape characteristic|
|inclination          |valid, planet's tilt relationship relative to earth|
|mass.massValue       |not valid, is a function of other features|
|mass.massExponent    |not valid, is a function of other features|
|vol.volValue         |not valid, is a function of other features|
|vol.volExponent      |not valid, is a function of other features|
|density              |valid, distinct feature of planet|
|escape               |valid, escape velocity is distinct feature of planet|
|meanRadius           |valid, distinct feature of planet|
|equaRadius           |valid, distinct feature of planet|
|polarRadius          |valid, distinct feature of planet|
|flattening           |valid, distinct feature of planet|
|dimension            |not valid since it's a string with 3 values delimited by x where order cannot be guaranteed so unable to compare|
|sideralOrbit         |valid, distinct feature of planet|
|sideralRotation      |valid, distinct feature of planet|
|aroundPlanet         |not valid, all values are null|
|discoveredBy         |not valid, knowing the discoverer will not help determine the other similar attributes|
|discoveryDate        |not valid, knowing the discover date will not help determine the other similar attributes|
|alternativeNamee     |not valid, knowing the alternative name will not help determining the other similar attributes|
|axialTilt            |valid, distinct feature of planet|


In [ ]:
# creating an array of fields that are suitable for input per the table above
suitable_fields = ['semimajorAxis', 'perihelion', 'aphelion', 'eccentricity', 'inclination', 'density', 'gravity', 
                   'escape', 'meanRadius', 'equaRadius', 'polarRadius', 'flattening', 'sideralOrbit', 'sideralRotation', 'axialTilt']

###Please use the following cell to populate the dictionary with the inputs that you plan to define.  
####PLEASE UNCOMMENT FIELDS YOU PLAN TO USE AND PROVIDE VALUES

In [ ]:
 # create a dictionary for the inputs based on what the players define.  PLEASE UNCOMMENT FIELDS YOU PLAN TO USE AND PROVIDE VALUES
input_dict = {#'semimajorAxis': 0, 
              'perihelion': 2700000000, 
              #'aphelion': 0,
              'eccentricity': 0.19126, 
              #'inclination': 0, 
              'density': 1.3, 
              #'gravity': 0,
              #'escape': 0, 
              #'meanRadius': 0,
              #'equaRadius': 0, 
              'polarRadius': 230, 
              #'flattening': 0, 
              #'sideralOrbit': 0, 
              #'sideralRotation': 0, 
              #'axialTilt': 0 
}

###Version 1: Deterministic Matching
####This will look for exact matches between your input per input_dict and planet values from the Solar System REST API.  Whichever planet has the most exact matches will be provided as output

Run this cell to establish the function


In [ ]:
# create function that will take a dictionary as input parameter and output the planet from the database that most closely matches the inputs provided based on exact matches
def get_closest_planet_deterministic(input_dict, suitable_fields):  
  
  # create dictionary for count of deterministic matches by planet index
  matched_count = {0: 0,
                   1: 0,
                   2: 0,
                   3: 0,
                   4: 0,
                   5: 0,
                   6: 0,
                   7: 0,
                   8: 0,
                   9: 0,
                   10: 0,
                   11: 0,
                   12: 0
                   }
  
  # create array to populate index of keys for matched values per data point
  match_ids_arr = []
  
  # create variable for the winning index, which will be used to output results
  matched_winner_index = 0

  # need to loop through all dictionary values to evaluate matching fields in DB
  for key in input_dict: 
    
    # if statement to evaluate if the key provided on input is part of suitable fields the game evaluates on
    if(key not in suitable_fields):
      print("The following input field is not supported in this game: " + key)
    
    else:
      print("input key:", key, "input value:", input_dict[key], "match id:", df[df[key]==input_dict[key]].index.values)  

      # add index to array when the input value matches a value from the DB
      match_ids_arr = df[df[key]==input_dict[key]].index.values
      
      # loop through the array of matches
      for index in match_ids_arr:
        # increment the index that matched in matched_count dictionary
        matched_count[index] += 1
        
      # reset match_ids_arr until next key is evaluated
      match_ids_arr = []
      
  # choose maximum match count as the winner
  maximum = max(matched_count, key=matched_count.get)
  matched_winner_index = maximum
  print("Thanks for playing!  Here's the planet closest to your defined inputs:") # print result message
  print(df.iloc[[matched_winner_index]]) # print result message

Run this cell to evaluate the function and receive output


In [ ]:
get_closest_planet_deterministic(input_dict, suitable_fields)

input key: perihelion input value: 2700000000 match id: []
input key: eccentricity input value: 0.19126 match id: [5]
input key: density input value: 1.3 match id: []
input key: polarRadius input value: 230 match id: []
Thanks for playing!  Here's the planet closest to your defined inputs:
       id             name  ... alternativeName  axialTilt
5  haumea  (136108) Hauméa  ...       2003 EL61        0.0

[1 rows x 27 columns]


###Version 2: Probabilistic Matching
####This will look for the difference in values between your input per input_dict and planet values from the Solar System REST API.  Whichever planet has the lowest absolute value summation will be provided as output.

Run this cell to establish the function

In [ ]:
# create function that will take a dictionary as input parameter and output the planet from the database that most closely matches the inputs provided based on approximate matches
def get_closest_planet_probabilistic(input_dict, suitable_fields):  
  
  # create dataframe for an absoluate value count of difference between provided values and planets' values
  df_evalmatrix = pd.DataFrame(np.zeros((15, 13)),index=suitable_fields, columns=[df['id'].to_numpy()])
  
  # create variable for the winning index, which will be used to output results
  matched_winner_index = 0

  # need to loop through all dictionary values to evaluate matching fields in DB
  for key in input_dict: 
    
    # if statement to evaluate if the key provided on input is part of suitable fields the game evaluates on
    if(key not in suitable_fields):
      print("The following input field is not supported in this game: " + key)
    
    else:
      print("input key:", key, "input value:", input_dict[key])  

      # loop through input_dict to calculate differences in values for each planet for each data point
      for count in range(len(df)):
        # subtract the value of the planet attribute from the value of the input attribute and take absolute value, increment this result to the appropriate cell in the weighting matrix
        df_evalmatrix.loc[key][count] += abs(df[key].values[count] - input_dict[key])

  # sum each column of the dataframe
  planet_sum = df_evalmatrix.sum(axis = 0, skipna = True) 

  # find the minimum sum of columns and choose that index as matched_winner
  matched_winner_index = df[df["id"]==planet_sum.idxmin()[0]].index[0]

  # print the matrix of weights
  print(df_evalmatrix)

  print("Thanks for playing!  The minimum match value was " , round(df_evalmatrix.sum(axis=0, skipna=True).min()), " and the maximum match value was ", round(df_evalmatrix.sum(axis=0, skipna=True).max()))
  print("Here's the planet closest to your defined inputs using probabilistic matching:") # print result message
  print(df.iloc[[matched_winner_index]]) # print result message

Run this cell to evaluate the function and receive output

In [ ]:
get_closest_planet_probabilistic(input_dict, suitable_fields)

input key: perihelion input value: 2700000000
input key: eccentricity input value: 0.19126
input key: density input value: 1.3
input key: polarRadius input value: 230
                        ceres          eris  ...         terre         venus
semimajorAxis    0.000000e+00  0.000000e+00  ...  0.000000e+00  0.000000e+00
perihelion       2.317380e+09  3.065733e+09  ...  2.552905e+09  2.592523e+09
aphelion         0.000000e+00  0.000000e+00  ...  0.000000e+00  0.000000e+00
eccentricity     1.154400e-01  2.494200e-01  ...  1.745600e-01  1.845600e-01
inclination      0.000000e+00  0.000000e+00  ...  0.000000e+00  0.000000e+00
density          8.610000e-01  1.220000e+00  ...  4.213600e+00  3.943000e+00
gravity          0.000000e+00  0.000000e+00  ...  0.000000e+00  0.000000e+00
escape           0.000000e+00  0.000000e+00  ...  0.000000e+00  0.000000e+00
meanRadius       0.000000e+00  0.000000e+00  ...  0.000000e+00  0.000000e+00
equaRadius       0.000000e+00  0.000000e+00  ...  0.000000e+00 